[View in Colaboratory](https://colab.research.google.com/github/MChamith/SpatioAE/blob/master/trainae.ipynb)

In [1]:
!git clone https://github.com/MChamith/SpatioAE

Cloning into 'SpatioAE'...
remote: Counting objects: 7091, done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 7091 (delta 6), reused 0 (delta 0), pack-reused 7076
Receiving objects: 100% (7091/7091), 219.96 MiB | 26.30 MiB/s, done.
Resolving deltas: 100% (22/22), done.
Checking out files: 100% (7204/7204), done.


In [0]:

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Activation, BatchNormalization
from keras.models import Model
from keras.preprocessing import image
from keras import backend as K
import cv2
import numpy as np
import os
from tqdm import tqdm
import fnmatch
import scipy.misc

TRAIN_DIR ='SpatioAE/Train'
TEST_DIR = 'SpatioAE/Test'
IMG_SIZE = 128



def create_training_set():
  train_data = []
  matches = []
  for root, dirnames, filenames in tqdm(os.walk(TRAIN_DIR)):
    for filename in fnmatch.filter(filenames, "*.tif"):
      path = os.path.join(root, filename)
      img = image.load_img(path, target_size=(128,128), grayscale = True)
      img_array = image.img_to_array(img)
      print("img array size is " + str(img_array.shape))
#       img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE) )
#       img_array = np.expand_dims(img_array, axis = 2)
      train_data.append(img_array)
          
  np.save('training_data.npy' , train_data)
  
  return train_data

      
    
    
#   for img in tqdm(os.listdir(TRAIN_DIR)):
#     path = os.path.join(TRAIN_DIR, img)
#     img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE) )
#     img = np.expand_dims(img, axis = 2)
#     train_data.append(np.array(img))
    
    
#   np.save('training_data.npy' , train_data)
  
#   return train_data


def create_test_set():
  test_data = []
  for img in tqdm(os.listdir(TEST_DIR)):
    path = os.path.join(TEST_DIR, img)
    img = image.load_img(path, target_size=(128,128), grayscale = True)
    img_array =image.img_to_array(img)
    
#     img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE) )
#     img = np.expand_dims(img, axis = 2)
    test_data.append(np.array(img))
    
    
  np.save('test_data.npy' , test_data)
  return test_data

In [0]:

def model_autoencoder():
  
  input_img  = Input(shape = (128,128,1))  #channel last
  
  x = Conv2D(filters = 32, kernel_size = (3,3),strides = (1,1) , padding ='same' , kernel_initializer = 'glorot_uniform')(input_img)
  x = BatchNormalization(axis = 3)(x)
  x = Activation('relu')(x)
  x = MaxPooling2D(pool_size = (2,2), strides = (2,2))(x)
  print("layer one conv " + str(x.shape))
  
  x = Conv2D(filters = 48, kernel_size = (3,3),strides = (1,1) , padding ='same' , kernel_initializer = 'glorot_uniform')(x)
  x = BatchNormalization(axis = 3)(x)
  x = Activation('relu')(x)
  x = MaxPooling2D(pool_size = (2,2), strides = (2,2))(x)
  print("layer two conv " + str(x.shape))
  
 
  x = Conv2D(filters = 64, kernel_size = (3,3),strides = (1,1) , padding ='same' , kernel_initializer = 'glorot_uniform')(x)
  x = BatchNormalization(axis = 3)(x)
  x = Activation('relu')(x)
  x = MaxPooling2D(pool_size = (2,2), strides = (2,2))(x)
  print("layer three conv " + str(x.shape))
  
  x = Conv2D(filters = 64, kernel_size = (3,3),strides = (1,1) , padding ='same' , kernel_initializer = 'glorot_uniform')(x)
  x = BatchNormalization(axis = 3)(x)
  x = Activation('relu')(x)
  encoded = MaxPooling2D(pool_size = (1,1), strides = (1,1))(x)
  print("layer four conv " + str(encoded.shape))
  
  
  ## decoder branch
  
  x = Conv2D(filters = 48, kernel_size = (3,3),strides = (1,1) , padding ='same' , kernel_initializer = 'glorot_uniform')(encoded)
  x = UpSampling2D(size = (2,2))(x)
  x = BatchNormalization(axis = 3)(x)
  x = Activation('relu')(x)
  print("layer one deconv " + str(x.shape))
 
  x = Conv2D(filters = 32, kernel_size = (3,3),strides = (1,1) , padding ='same' , kernel_initializer = 'glorot_uniform')(x)
  x = UpSampling2D(size = (2,2))(x)
  x = BatchNormalization(axis = 3)(x)
  x = Activation('relu')(x)
  print("layer two deconv " + str(x.shape))
  
  x = Conv2D(filters = 32, kernel_size = (3,3),strides = (1,1) , padding ='same' , kernel_initializer = 'glorot_uniform')(x)
  x = UpSampling2D(size = (2,2))(x)
  x = BatchNormalization(axis = 3)(x)
  x = Activation('relu')(x)
  print("layer three deconv " + str(x.shape))
  decoded = Conv2D(filters = 1, kernel_size = (3,3), strides = (1,1), padding = 'same', activation = 'sigmoid')(x)
  print("output" + str(decoded.shape))

  
  
  autoencoder = Model(input_img, decoded)
  
  return autoencoder
  

In [13]:
get_data = create_training_set()
input_imgs = np.load('training_data.npy')
input_imgs = input_imgs.astype('float32') / 255.
print(input_imgs.shape)
K.clear_session()
autoencoder = model_autoencoder()
autoencoder.compile(optimizer = 'adam' , loss = 'mean_squared_error')

autoencoder.fit(input_imgs,input_imgs,  epochs =50, batch_size= 100)
  
    

0it [00:00, ?it/s]


(0,)
layer one conv (?, 64, 64, 32)
layer two conv (?, 32, 32, 48)
layer three conv (?, 16, 16, 64)
layer four conv (?, 16, 16, 64)
layer one deconv (?, 32, 32, 48)
layer two deconv (?, 64, 64, 32)
layer three deconv (?, 128, 128, 32)
output(?, 128, 128, 1)


ValueError: ignored

In [14]:
get_test_data = create_test_set()
test_imgs = np.load('test_data.npy')
test_imgs = input_imgs.astype('float32') / 255.
print(input_imgs.shape)

decode_imgs = autoencoder.predict(test_imgs)

FileNotFoundError: ignored

In [1]:
cv2.imshow('test img', np.squeeze(test_imgs[1], axis = 2))

NameError: ignored

In [2]:
import matplotlib.pyplot as plt
test_data = []
for img in tqdm(os.listdir(TEST_DIR)):
  path = os.path.join(TEST_DIR, img)
  temp_img = cv2.resize(cv2.imread(path,0),(IMG_SIZE,IMG_SIZE) )
  cv2.imshow('temp',temp_img)
  print(temp_img.shape)
  
  temp_img = np.expand_dims(temp_img, axis = 2)
  
  print(temp_img.shape)
#   plt.imshow(np.squeeze(temp_img, axis = 2))
  test_data.append(np.array(temp_img))
  

NameError: ignored